# Data Prep -- Extracting CWD road types

In [1]:
import os, sys

import numpy as np
from numpy import random

import rasterio
from rasterio import features
from rasterio import transform
from rasterio.transform import Affine

import pandas as pd
import geopandas as gpd

import shapely
from shapely.geometry import *
from shapely.wkt import loads
from shapely.ops import transform

sys.path.append('../../src/')

from gostrocks.src.GOSTRocks.misc import tPrint

/Users/robert/.pyenv/versions/geo/lib/python3.9/site-packages/geopandas/_compat.py:106: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.0-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


### Setup

File paths

In [2]:
data_dir = r'../../data'

rd_dir = r'roads'
base_dir = r'rast_inputs'

speed_dir = r'speed'

### Import tables

In [3]:
cwd = pd.read_csv(os.path.join(data_dir,rd_dir,'cwd_inventory_raw.csv'))

In [4]:
cwd.head(2)

,#,District,S/No,Road_Num,Road_Name,Road_Class_Code,Road_Class,Road_length,Pavement_Type,Year_of_Construcation,Status_Condition,Traffic_Load_(_Low._Mid.High),Carriageway_Width,Number_of_Lane,Seg_Name,RONET_Traffic_Category,RONET_Surface_Type
0,65,Abbottabad,65.0,NaN,Maira Rehmat Khan Kanthiali Road,1.0,District Road,37.0,AC,1996,Good,High,5,2,NaN,Traffic-V,Asphaltic Mix
1,42,Abbottabad,42.0,NaN,Fowara Chowk to Xen C&W Residence,NaN,Access Road,2.8,AC,1970,Good,High,7.3,2,NaN,Traffic-IV,Asphaltic Mix


In [5]:
# elev = pd.read_csv(os.path.join(data_dir,rd_dir,'combined_kpk_mnt_splt_utm.csv'))
elev = gpd.read_file(os.path.join(data_dir,rd_dir,'Pak_Official_Roads_Combined_210826/combined_kpk_mnt_splt_utm.shp'))

In [6]:
elev.head(2)

,fid_1,cat,fid_12,Road_ID,Road_No,Road_Name,SegNo,Seg_Name,Road_Class,Pavement_T,...,MNT_RONE_1,MNT_RONE_2,MNT_Functy,MNT_Traf_1,MNT_Surfac,MNT_Condit,MNT_Final_,Z_Mean,Avg_Slope,geometry
0,1.0,1,1.0,1,S-1,Peshawar-Charsadda-Mardan-Swabi-Topi- Ghazi-Si...,1.0,"Peshawar,Charsadda Road(Dual carriage Way)",PKHA,Asphalt Concrete,...,None,None,None,None,None,None,None,301.968822,2.888695,"LINESTRING (753230.065 3782493.432, 753206.004..."
1,2.0,1,1.0,1,S-1,Peshawar-Charsadda-Mardan-Swabi-Topi- Ghazi-Si...,1.0,"Peshawar,Charsadda Road(Dual carriage Way)",PKHA,Asphalt Concrete,...,None,None,None,None,None,None,None,297.456491,2.627899,"LINESTRING (751720.466 3782845.841, 751693.010..."


### Analyze the data

#### Elevation and winterization data analysis

In [7]:
elev.columns

Index(['fid_1', 'cat', 'fid_12', 'Road_ID', 'Road_No', 'Road_Name', 'SegNo',
       'Seg_Name', 'Road_Class', 'Pavement_T', 'Avg_Right', 'BT_Width',
       'Shoulder_T', 'Shoulder_W', 'Road_Lengt', 'No_of_Brid', 'No_of_Culv',
       'TYPE', 'Year_of_Co', 'Contractor', 'Executing', 'Remarks', 'Name',
       'Division', 'SHAPE_Leng', 'District', 'ADP', 'ADP_Serial', 'Sub_Scheme',
       'Search_Nam', 'Search_Num', 'road', 'x_cor', 'y_Cor', 'ID',
       'ROADS_Join', 'MNT_Num', 'MNT_Distri', 'MNT_S_No', 'MNT_Road_N',
       'MNT_Road_1', 'MNT_Road_C', 'MNT_Road_2', 'MNT_Road_l', 'MNT_Paveme',
       'MNT_Year_o', 'MNT_RONET_', 'MNT_Traffi', 'MNT_Carria', 'MNT_Number',
       'MNT_Seg_Na', 'MNT_RONE_1', 'MNT_RONE_2', 'MNT_Functy', 'MNT_Traf_1',
       'MNT_Surfac', 'MNT_Condit', 'MNT_Final_', 'Z_Mean', 'Avg_Slope',
       'geometry'],
      dtype='object')

In [8]:
elev.TYPE.unique()

array(['All Weather', None, 'Snow Bound', 'B', 'A', 'All weaher', 'A.W',
       'S.B', 'All weather'], dtype=object)

In [9]:
aw_dct = dict.fromkeys(['All Weather',np.nan,'A','All weaher','A.W.','All weather'], 'All Weather')
sb_dct = dict.fromkeys(['S.B.','Snow Bound'], 'Snow Bound')

In [10]:
season_dct = {}
season_dct.update(aw_dct)
season_dct.update(sb_dct)

In [11]:
elev['Seasonality'] = elev.TYPE.map(season_dct)

In [12]:
elev.Seasonality.unique()

array(['All Weather', nan, 'Snow Bound'], dtype=object)

In [14]:
elev

,fid_1,cat,fid_12,Road_ID,Road_No,Road_Name,SegNo,Seg_Name,Road_Class,Pavement_T,...,MNT_RONE_2,MNT_Functy,MNT_Traf_1,MNT_Surfac,MNT_Condit,MNT_Final_,Z_Mean,Avg_Slope,geometry,Seasonality
0,1.0,1,1.0,1,S-1,Peshawar-Charsadda-Mardan-Swabi-Topi- Ghazi-Si...,1.0,"Peshawar,Charsadda Road(Dual carriage Way)",PKHA,Asphalt Concrete,...,None,None,None,None,None,None,301.968822,2.888695,"LINESTRING (753230.065 3782493.432, 753206.004...",All Weather
1,2.0,1,1.0,1,S-1,Peshawar-Charsadda-Mardan-Swabi-Topi- Ghazi-Si...,1.0,"Peshawar,Charsadda Road(Dual carriage Way)",PKHA,Asphalt Concrete,...,None,None,None,None,None,None,297.456491,2.627899,"LINESTRING (751720.466 3782845.841, 751693.010...",All Weather
2,3.0,1,1.0,1,S-1,Peshawar-Charsadda-Mardan-Swabi-Topi- Ghazi-Si...,1.0,"Peshawar,Charsadda Road(Dual carriage Way)",PKHA,Asphalt Concrete,...,None,None,None,None,None,None,297.611728,3.436315,"LINESTRING (750113.667 3781854.106, 750071.618...",All Weather
3,4.0,1,1.0,1,S-1,Peshawar-Charsadda-Mardan-Swabi-Topi- Ghazi-Si...,1.0,"Peshawar,Charsadda Road(Dual carriage Way)",PKHA,Asphalt Concrete,...,None,None,None,None,None,None,297.079688,2.529004,"LINESTRING (748578.407 3780675.967, 748529.082...",All Weather
4,5.0,1,1.0,1,S-1,Peshawar-Charsadda-Mardan-Swabi-Topi- Ghazi-Si...,1.0,"Peshawar,Charsadda Road(Dual carriage Way)",PKHA,Asphalt Concrete,...,None,None,None,None,None,None,298.843813,1.714421,"LINESTRING (747415.585 3779232.688, 747411.570...",All Weather
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10055,10056.0,3237,3237.0,None,None,140942-Rehabilitation of Pandu road Peshawar (...,0.0,None,None,None,...,None,None,None,None,None,None,358.312507,1.752502,"LINESTRING (752232.966 3757324.827, 752223.162...",NaN
10056,10057.0,3237,3237.0,None,None,140942-Rehabilitation of Pandu road Peshawar (...,0.0,None,None,None,...,None,None,None,None,None,None,340.771399,2.422172,"LINESTRING (752203.133 3758641.754, 752192.956...",NaN
10057,10058.0,3238,3238.0,None,None,"140997-F/S, Design and Construction/Rehabilita...",0.0,None,None,None,...,None,None,None,None,None,None,306.040107,2.955499,"LINESTRING (740684.478 3775606.958, 740679.060...",NaN
10058,10059.0,3238,3238.0,None,None,"140997-F/S, Design and Construction/Rehabilita...",0.0,None,None,None,...,None,None,None,None,None,None,309.893128,2.207062,"LINESTRING (739224.749 3775796.185, 739213.927...",NaN


In [15]:
elev['length'] = elev.geometry.length

In [17]:
sum(elev.length)/1000

16523.770381353166

**Now pivot the data**

In [141]:

elev['Terrain'] = pd.cut(elev['Avg_Slope'], [-np.inf, 8, 16, np.inf], 
                           labels = ['Plains', 'Hills', 'Mountains']) # change labels herelabels = ['Plains', 'Hills', 'Mountains']) # change labels here
                           
elev_v_classcond = elev.pivot_table(index=['Terrain','Road_Class'],\
                               values='length',\
                               aggfunc='sum')

elev_v_classcond['Pct_of_Terrain_total'] = round(elev_v_classcond.groupby(['Terrain']).length.apply(lambda x: x/x.sum() * 100),2)

In [142]:
# not very illuminating
elev_v_classcond

length  Pct_of_Terrain_total
Terrain   Road_Class                                    
Plains    A           2.652427e+06                 31.77
          B           4.194394e+06                 50.25
          PKHA        1.497059e+06                 17.93
          S           3.813740e+03                  0.05
Hills     A           8.826136e+05                 28.48
          B           1.817612e+06                 58.65
          PKHA        3.833385e+05                 12.37
          S           1.525496e+04                  0.49
Mountains A           8.813339e+05                 37.80
          B           1.276965e+06                 54.76
          PKHA        1.658000e+05                  7.11
          S           7.627481e+03                  0.33

In [143]:
elev['Elevation_meters'] = pd.cut(elev['Z_Mean'], [-np.inf, 500, 1000, 1500, 1750, 2000, 2100, 2200, 2300, 2400, 2500,\
                                                       2600, 2750, 3000, 3500, 4000, np.inf], 
                          
                           labels = ['0 - 500', '501 - 1000', '1001 - 1500', '1501 - 1750','1751 - 2000',\
                                     '2001 - 2100','2101 - 2200','2201 - 2300','2301 - 2400','2401 - 2500',\
                                     '2501 - 2600','2601 - 2750','2751 - 3000','3001 - 3500','3501 - 4000','4001+']) # change labels herelabels = ['Plains', 'Hills', 'Mountains']) # change labels here

In [144]:
elev_v_snow = elev.pivot_table(index=['Elevation_meters','Seasonality'],\
                               values='length',\
                               aggfunc='sum')

elev_v_snow['Pct_of_Elevation_Category_total'] = round(elev_v_snow.groupby(['Elevation_meters']).length.apply(lambda x: x/x.sum() * 100),2)

In [145]:
elev_v_snow

length  Pct_of_Elevation_Category_total
Elevation_meters Seasonality                                               
0 - 500          All Weather  5.198593e+06                           100.00
                 Snow Bound   0.000000e+00                             0.00
501 - 1000       All Weather  3.762098e+06                           100.00
                 Snow Bound   0.000000e+00                             0.00
1001 - 1500      All Weather  2.707527e+06                            95.77
                 Snow Bound   1.197297e+05                             4.23
1501 - 1750      All Weather  6.122740e+05                            85.13
                 Snow Bound   1.069363e+05                            14.87
1751 - 2000      All Weather  2.624951e+05                            65.22
                 Snow Bound   1.399759e+05                            34.78
2001 - 2100      All Weather  4.639277e+04                            66.81
                 Snow Bound   2.304528e+04                            33.19
2101 - 2200      All Weather  3.244418e+04                            61.71
                 Snow Bound   2.013444e+04                            38.29
2201 - 2300      All Weather  2.187133e+04                            29.24
                 Snow Bound   5.293002e+04                            70.76
2301 - 2400      All Weather  2.151470e+04                            19.15
                 Snow Bound   9.082542e+04                            80.85
2401 - 2500      All Weather  1.926291e+03                             4.46
                 Snow Bound   4.122188e+04                            95.54
2501 - 2600      All Weather  0.000000e+00                             0.00
                 Snow Bound   2.549081e+04                           100.00
2601 - 2750      All Weather  0.000000e+00                             0.00
                 Snow Bound   2.155479e+04                           100.00
2751 - 3000      All Weather  0.000000e+00                             0.00
                 Snow Bound   1.959072e+04                           100.00
3001 - 3500      All Weather  0.000000e+00                             0.00
                 Snow Bound   1.567258e+04                           100.00
3501 - 4000      All Weather  0.000000e+00                             0.00
                 Snow Bound   7.836288e+03                           100.00
4001+            All Weather  0.000000e+00                              NaN
                 Snow Bound   0.000000e+00                              NaN

In [146]:
elev_v_snow.to_csv(os.path.join(data_dir,rd_dir,'elev_snowbound_breakdown.csv'))

### Pavement and condition + elevation analysis

In [147]:
elev.columns

Index(['fid_1', 'cat', 'fid_12', 'Road_ID', 'Road_No', 'Road_Name', 'SegNo',
       'Seg_Name', 'Road_Class', 'Pavement_T', 'Avg_Right', 'BT_Width',
       'Shoulder_T', 'Shoulder_W', 'Road_Lengt', 'No_of_Brid', 'No_of_Culv',
       'TYPE', 'Year_of_Co', 'Contractor', 'Executing', 'Remarks', 'Name',
       'Division', 'SHAPE_Leng', 'District', 'ADP', 'ADP_Serial', 'Sub_Scheme',
       'Search_Nam', 'Search_Num', 'road', 'x_cor', 'y_Cor', 'ID',
       'ROADS_Join', 'MNT_Num', 'MNT_Distri', 'MNT_S_No', 'MNT_Road_N',
       'MNT_Road_1', 'MNT_Road_C', 'MNT_Road_2', 'MNT_Road_l', 'MNT_Paveme',
       'MNT_Year_o', 'MNT_RONET_', 'MNT_Traffi', 'MNT_Carria', 'MNT_Number',
       'MNT_Seg_Na', 'MNT_RONE_1', 'MNT_RONE_2', 'MNT_Functy', 'MNT_Traf_1',
       'MNT_Surfac', 'MNT_Condit', 'MNT_Final_', 'Z_Mean', 'Avg_Slope',
       'geometry', 'Seasonality', 'length', 'Terrain', 'Elevation_meters'],
      dtype='object')

Cleanup

In [148]:
elev.rename({'MNT_RONET_' : 'Status_Condition',\
             'MNT_RONE_2' : 'Surface',\
             'MNT_Road_2' : 'Road_Class_RONET'},\
            axis=1,inplace=True)

In [149]:
elev.Road_Class_RONET = elev.Road_Class_RONET.replace({'District road' : 'District Road', 'Collector road' : 'Collector Road', 'Arterial Road' : 'District Road'})
elev.Status_Condition = elev.Status_Condition.replace({'poor' : 'Poor'})

In [150]:
elev['Status_Condition'] = pd.Categorical(elev['Status_Condition'], ["Poor", "Fair", "Good"])

Pivoting

In [151]:
elev_rclass = elev.pivot_table(index=['Terrain','Road_Class_RONET'],\
                        values='length',
                        aggfunc='sum')

elev_cond = elev.pivot_table(index=['Terrain','Status_Condition'],\
                        values='length',
                        aggfunc='sum')

elev_pave_cond = elev.pivot_table(index=['Terrain','Surface','Status_Condition'],\
                        values='length',
                        aggfunc='sum')
                          
elev_rclass_cond = elev.pivot_table(index=['Terrain','Road_Class_RONET','Status_Condition'],\
                            values='length',
                            aggfunc='sum')

elev_rclass['Road_Class_as_pct_of_Terrain_total'] = round(elev_rclass.groupby(['Terrain']).length.apply(lambda x: x/x.sum() * 100),2)
elev_cond['Status_as_pct_of_Terrain_total'] = round(elev_cond.groupby(['Terrain']).length.apply(lambda x: x/x.sum() * 100),2)
elev_pave_cond['Status_as_pct_of_Surface_total'] = round(elev_pave_cond.groupby(['Terrain','Surface']).length.apply(lambda x: x/x.sum() * 100),2)
elev_rclass_cond['Status_as_pct_of_Class_total'] = round(elev_rclass_cond.groupby(['Terrain','Road_Class_RONET']).length.apply(lambda x: x/x.sum() * 100),2)

In [152]:
elev_rclass

length  Road_Class_as_pct_of_Terrain_total
Terrain   Road_Class_RONET                                                  
Plains    Access Road       1.166293e+06                               34.93
          Collector Road    6.421926e+05                               19.23
          District Road     1.530503e+06                               45.84
Hills     Access Road       6.313205e+05                               40.16
          Collector Road    2.997206e+04                                1.91
          District Road     9.107137e+05                               57.93
Mountains Access Road       4.966904e+05                               41.93
          Collector Road    2.020673e+04                                1.71
          District Road     6.676599e+05                               56.36

In [153]:
elev_cond

length  Status_as_pct_of_Terrain_total
Terrain   Status_Condition                                              
Plains    Poor              1.546735e+06                           46.32
          Fair              7.342894e+05                           21.99
          Good              1.057964e+06                           31.69
Hills     Poor              6.437357e+05                           40.95
          Fair              5.013695e+05                           31.89
          Good              4.269011e+05                           27.16
Mountains Poor              6.156116e+05                           51.97
          Fair              3.346961e+05                           28.25
          Good              2.342493e+05                           19.78

In [157]:
elev_pave_cond

length  \
Terrain   Surface           Status_Condition                  
Plains    Asphaltic Mix     Poor              226643.791112   
                            Fair              236994.136211   
                            Good              454322.188199   
          Cement Concrete   Poor               31308.580829   
                            Fair                5902.631562   
                            Good               15033.667756   
          Earth             Poor                4204.869603   
                            Fair                   0.000000   
                            Good                3224.411495   
          Gravel            Poor              392822.020657   
                            Fair               26179.279629   
                            Good              110049.407797   
          Surface Treatment Poor              891756.147091   
                            Fair              465213.397082   
                            Good              475334.084692   
Hills     Asphaltic Mix     Poor              151436.540034   
                            Fair              219556.078977   
                            Good              305143.785600   
          Cement Concrete   Poor               29622.161353   
                            Fair               11713.244653   
                            Good                8333.805440   
          Earth             Poor                   0.000000   
                            Fair                   0.000000   
                            Good                   0.000000   
          Gravel            Poor              319685.128561   
                            Fair               84495.056361   
                            Good               16840.033767   
          Surface Treatment Poor              142991.871761   
                            Fair              185605.108799   
                            Good               96583.454559   
Mountains Asphaltic Mix     Poor              133687.281174   
                            Fair              115992.364418   
                            Good              187383.502476   
          Cement Concrete   Poor               23078.632185   
                            Fair                7686.372403   
                            Good                6922.384178   
          Earth             Poor                   0.000000   
                            Fair                   0.000000   
                            Good                   0.000000   
          Gravel            Poor              407127.258955   
                            Fair               81355.650765   
                            Good               16580.710689   
          Surface Treatment Poor               51718.401374   
                            Fair              129661.724487   
                            Good               23362.717269   

                                              Status_as_pct_of_Surface_total  
Terrain   Surface           Status_Condition                                  
Plains    Asphaltic Mix     Poor                                       24.69  
                            Fair                                       25.82  
                            Good                                       49.49  
          Cement Concrete   Poor                                       59.93  
                            Fair                                       11.30  
                            Good                                       28.78  
          Earth             Poor                                       56.60  
                            Fair                                        0.00  
                            Good                                       43.40  
          Gravel            Poor                                       74.25  
                            Fair                                        4.95  
                            Good                                

In [158]:
elev_rclass_cond

length  \
Terrain   Road_Class_RONET Status_Condition                  
Plains    Access Road      Poor              520362.462716   
                           Fair              369871.211571   
                           Good              276059.741880   
          Collector Road   Poor              323406.797837   
                           Fair               11308.940115   
                           Good              307476.877362   
          District Road    Poor              702966.148739   
                           Fair              353109.292798   
                           Good              474427.140697   
Hills     Access Road      Poor              268475.873671   
                           Fair              223612.491406   
                           Good              139232.152915   
          Collector Road   Poor               11904.205388   
                           Fair                   0.000000   
                           Good               18067.857718   
          District Road    Poor              363355.622651   
                           Fair              277756.997385   
                           Good              269601.068734   
Mountains Access Road      Poor              262679.990345   
                           Fair              154648.353079   
                           Good               79362.046178   
          Collector Road   Poor               10676.045792   
                           Fair                   0.000000   
                           Good                9530.682875   
          District Road    Poor              342255.537550   
                           Fair              180047.758994   
                           Good              145356.585559   

                                             Status_as_pct_of_Class_total  
Terrain   Road_Class_RONET Status_Condition                                
Plains    Access Road      Poor                                     44.62  
                           Fair                                     31.71  
                           Good                                     23.67  
          Collector Road   Poor                                     50.36  
                           Fair                                      1.76  
                           Good                                     47.88  
          District Road    Poor                                     45.93  
                           Fair                                     23.07  
                           Good                                     31.00  
Hills     Access Road      Poor                                     42.53  
                           Fair                                     35.42  
                           Good                                     22.05  
          Collector Road   Poor                                     39.72  
                           Fair                                      0.00  
                           Good                                     60.28  
          District Road    Poor                                     39.90  
                           Fair                                     30.50  
                           Good                                     29.60  
Mountains Access Road      Poor                                     52.89  
                           Fair                                     31.14  
                           Good                                     15.98  
          Collector Road   Poor                                     52.83  
                           Fair                                      0.00  
                           Good                                     47.17  
          District Road    Poor                                     51.26  
                           Fair                                     26.97  
                           Good                                     21.77

Export

In [159]:
elev_rclass.to_csv(os.path.join(data_dir,rd_dir,'Pivot_Tables/elev_rclass.csv'))
elev_cond.to_csv(os.path.join(data_dir,rd_dir,'Pivot_Tables/elev_cond.csv'))
elev_pave_cond.to_csv(os.path.join(data_dir,rd_dir,'Pivot_Tables/elev_pave_cond.csv'))
elev_rclass_cond.to_csv(os.path.join(data_dir,rd_dir,'Pivot_Tables/elev_rclass_cond.csv'))

## CWD pavement and condition analysis

In [165]:
cwd.columns

Index(['#', 'District', 'S/No', 'Road_Num', 'Road_Name', 'Road_Class_Code',
       'Road_Class', 'Road_length', 'Pavement_Type', 'Year_of_Construcation',
       'Status_Condition', 'Traffic_Load_(_Low._Mid.High)',
       'Carriageway_Width', 'Number_of_Lane', 'Seg_Name',
       'RONET_Traffic_Category', 'RONET_Surface_Type'],
      dtype='object')

In [198]:
cwd['Status_Condition'] = pd.Categorical(cwd['Status_Condition'], ["Poor", "Fair", "Good"])

In [216]:
cwd.Road_Class = cwd.Road_Class.str.title()

In [218]:
rclass = cwd.pivot_table(index=['Road_Class'],\
                        values='Road_length',
                        aggfunc='sum')

rclass_cond = cwd.pivot_table(index=['Road_Class','Status_Condition'],\
                            values='Road_length',
                            aggfunc='sum')

rclass['Pct_of_total'] = round((rclass.Road_length / rclass.Road_length.sum() * 100),2)
rclass_cond['Status_as_pct_of_Surface_total'] = round(rclass_cond.groupby(['Road_Class']).Road_length.apply(lambda x: x/x.sum() * 100),2)

In [219]:
rclass

,Road_length,Pct_of_total
Road_Class,,
Access Road,7341.23,33.86
Arterial Road,1730.31,7.98
Collector Road,4236.22,19.54
District Road,8371.13,38.61


In [220]:
rclass_cond

Road_length  Status_as_pct_of_Surface_total
Road_Class     Status_Condition                                             
Access Road    Poor                 3330.690                           45.41
               Fair                 2061.135                           28.10
               Good                 1943.405                           26.49
Arterial Road  Poor                  580.290                           33.54
               Fair                   13.880                            0.80
               Good                 1136.140                           65.66
Collector Road Poor                 2064.610                           48.74
               Fair                   35.980                            0.85
               Good                 2135.630                           50.41
District Road  Poor                 2666.740                           31.86
               Fair                 4095.870                           48.93
               Good                 1608.520                           19.22

In [199]:
pave = cwd.pivot_table(index=['RONET_Surface_Type'],\
                        values='Road_length',
                        aggfunc='sum')

pave_cond = cwd.pivot_table(index=['RONET_Surface_Type','Status_Condition'],\
                            values='Road_length',
                            aggfunc='sum')

pave['Pct_of_total'] = round((pave.Road_length / pave.Road_length.sum() * 100),2)
pave_cond['Status_as_pct_of_Surface_total'] = round(pave_cond.groupby(['RONET_Surface_Type']).Road_length.apply(lambda x: x/x.sum() * 100),2)

In [200]:
pave

,Road_length,Pct_of_total
RONET_Surface_Type,,
Asphaltic Mix,5560.565,25.65
Cement Concrete,732.810,3.38
Earth,25.220,0.12
Gravel,4145.360,19.12
Surface Treatment,11214.935,51.73


In [201]:
pave_cond

Road_length  \
RONET_Surface_Type Status_Condition                
Asphaltic Mix      Poor                 1487.810   
                   Fair                 1154.775   
                   Good                 2917.980   
Cement Concrete    Poor                  308.210   
                   Fair                   63.000   
                   Good                  361.600   
Earth              Poor                   12.000   
                   Fair                    0.000   
                   Good                   13.220   
Gravel             Poor                 2885.330   
                   Fair                  655.060   
                   Good                  604.970   
Surface Treatment  Poor                 3948.980   
                   Fair                 4334.030   
                   Good                 2925.925   

                                     Status_as_pct_of_Surface_total  
RONET_Surface_Type Status_Condition                                  
Asphaltic Mix      Poor                                       26.76  
                   Fair                                       20.77  
                   Good                                       52.48  
Cement Concrete    Poor                                       42.06  
                   Fair                                        8.60  
                   Good                                       49.34  
Earth              Poor                                       47.58  
                   Fair                                        0.00  
                   Good                                       52.42  
Gravel             Poor                                       69.60  
                   Fair                                       15.80  
                   Good                                       14.59  
Surface Treatment  Poor                                       35.23  
                   Fair                                       38.67  
                   Good                                       26.10

Now by district

In [202]:
dist_pave = cwd.pivot_table(index=['District','RONET_Surface_Type'],\
                            values='Road_length',
                            aggfunc='sum')

dist_pave_cond = cwd.pivot_table(index=['District','RONET_Surface_Type','Status_Condition'],\
                            values='Road_length',
                            aggfunc='sum')

dist_pave['Surface_type_as_pct_of_District_total'] = round(dist_pave.groupby(['District']).Road_length.apply(lambda x: x/x.sum() * 100),2)

dist_pave_cond['Surface_type_as_pct_of_District_total'] = round(dist_pave_cond.groupby(['District']).Road_length.apply(lambda x: x/x.sum() * 100),2)
dist_pave_cond['Status_as_pct_of_District_total'] = round(dist_pave_cond.groupby(['District']).Road_length.apply(lambda x: x/x.sum() * 100),2)
dist_pave_cond['Status_as_pct_of_District_Surface_total'] = round(dist_pave_cond.groupby(['District','RONET_Surface_Type']).Road_length.apply(lambda x: x/x.sum() * 100),2)

In [203]:
dist_pave

Road_length  \
District            RONET_Surface_Type                
Abbottabad          Asphaltic Mix            773.51   
                    Cement Concrete           74.00   
                    Gravel                   218.78   
                    Surface Treatment          5.30   
Bannu               Asphaltic Mix            223.32   
...                                             ...   
TD South Waziristan Surface Treatment        881.05   
Tank                Gravel                     7.70   
                    Surface Treatment        201.00   
Torghar             Gravel                    80.30   
                    Surface Treatment         75.40   

                                        Surface_type_as_pct_of_District_total  
District            RONET_Surface_Type                                         
Abbottabad          Asphaltic Mix                                       72.18  
                    Cement Concrete                                      6.91  
                    Gravel                                              20.42  
                    Surface Treatment                                    0.49  
Bannu               Asphaltic Mix                                       49.44  
...                                                                       ...  
TD South Waziristan Surface Treatment                                   66.88  
Tank                Gravel                                               3.69  
                    Surface Treatment                                   96.31  
Torghar             Gravel                                              51.57  
                    Surface Treatment                                   48.43  

[100 rows x 2 columns]

In [204]:
dist_pave_cond

Road_length  \
District   RONET_Surface_Type Status_Condition                
Abbottabad Asphaltic Mix      Poor                    83.10   
                              Fair                   229.60   
                              Good                   460.81   
           Cement Concrete    Poor                     0.00   
                              Fair                    29.80   
...                                                     ...   
Torghar    Gravel             Fair                     0.00   
                              Good                     0.00   
           Surface Treatment  Poor                     0.00   
                              Fair                    75.40   
                              Good                     0.00   

                                                Surface_type_as_pct_of_District_total  \
District   RONET_Surface_Type Status_Condition                                          
Abbottabad Asphaltic Mix      Poor                                               7.75   
                              Fair                                              21.43   
                              Good                                              43.00   
           Cement Concrete    Poor                                               0.00   
                              Fair                                               2.78   
...                                                                               ...   
Torghar    Gravel             Fair                                               0.00   
                              Good                                               0.00   
           Surface Treatment  Poor                                               0.00   
                              Fair                                              48.43   
                              Good                                               0.00   

                                                Status_as_pct_of_District_total  \
District   RONET_Surface_Type Status_Condition                                    
Abbottabad Asphaltic Mix      Poor                                         7.75   
                              Fair                                        21.43   
                              Good                                        43.00   
           Cement Concrete    Poor                                         0.00   
                              Fair                                         2.78   
...                                                                         ...   
Torghar    Gravel             Fair                                         0.00   
                              Good                                         0.00   
           Surface Treatment  Poor                                         0.00   
                              Fair                                        48.43   
                              Good                                         0.00   

                                                Status_as_pct_of_District_Surface_total  
District   RONET_Surface_Type Status_Condition                                           
Abbottabad Asphaltic Mix      Poor                                                10.74  
                              Fair                                                29.68  
                              Good                                                59.57  
           Cement Concrete    Poor                                                 0.00  
                              Fair                                                40.27  
...                                                                                 ...  
Torghar    Gravel             Fair                                                 0.00  
                              Good                                                 0.00  
           Surface Treatment  Poor                                                 0.00  


Export results

In [205]:
pave.to_csv(os.path.join(data_dir,rd_dir,'cwd_inventory_pivot_pave.csv'))
pave_cond.to_csv(os.path.join(data_dir,rd_dir,'cwd_inventory_pivot_pave_condition.csv'))

In [206]:
dist_pave.to_csv(os.path.join(data_dir,rd_dir,'cwd_inventory_pivot_dist_pave.csv'))
dist_pave_cond.to_csv(os.path.join(data_dir,rd_dir,'cwd_inventory_pivot_dist_pave_condition.csv'))